Make a Breakout Game!
===
本文将讲述如何借助于pygame,从头构建一个打砖块游戏。在这个练习中你将综合运用到面向对象的主要知识。

Your job in this assignment is to write the classic arcade game of
Breakout, which was invented by Steve Wozniak before he
founded Apple with Steve Jobs. It is a large assignment, but
entirely manageable as long as you break the problem up into
pieces. The decomposition is discussed in this handout, and
there are several suggestions for staying on top of things in the
“Strategy and tactics” section later in this handout.

游戏中使用到了以下模块和常量：
===

In [1]:

import sys
import pygame
from random import Random

# Object dimensions
SCREEN_SIZE   = 400,600

# Object dimensions

BRICK_HEIGHT  = 10
PADDLE_WIDTH  = 50
PADDLE_HEIGHT = 10
BALL_DIAMETER = 14
BALL_RADIUS   = int(BALL_DIAMETER/2)
NBRICKS_PER_ROW = 10
NBRICK_ROWS = 10
BRICK_SEP = 4
BRICK_OFFSET_Y=70
BRICK_WIDTH  = int((SCREEN_SIZE[0] - (NBRICKS_PER_ROW) * BRICK_SEP) / NBRICKS_PER_ROW)
MAX_PADDLE_X = SCREEN_SIZE[0] - PADDLE_WIDTH
MAX_BALL_X   = SCREEN_SIZE[0] - BALL_DIAMETER
MAX_BALL_Y   = SCREEN_SIZE[1] - BALL_DIAMETER

# Paddle Y coordinate
PADDLE_Y = SCREEN_SIZE[1] - PADDLE_HEIGHT - 10

# Color constants
BLACK = (0,0,0)
WHITE = (255,255,255)
RED=(255,0,0)
GREEN=(0,255,0)
BLUE  = (0,0,255)
BRICK_COLOR = (200,200,0)
ORANGE=(255,153,0)
BROWN=(255,204,0)
# State constants
STATE_BALL_IN_PADDLE = 0
STATE_PLAYING = 1
STATE_WON = 2
STATE_GAME_OVER = 3

开始写代码之前
===
对于打砖块游戏，我们都知道这个游戏的基本规则与玩法，那就是一个用球拍(paddle)接住运动的球(ball)，将其反弹，并击碎砖块得分，最终清完所有砖块后胜利，如果没有接住球，则减少一条生命，生命值到0以后就失败了。

鉴于上述要求，我们首先需要对程序进行构思：首先需要有对场景、球拍、球、砖块的描述，场景中球拍、球、砖块相互作用而实现游戏功能。因此我们可以建立如下几个类：Breakout(游戏运行类)，Paddle,Ball,Brick。

数据结构设计
===
游戏中对象实体和你所看到的图像并不是一回事，你所看到的游戏画面只是程序将后台的对象实体相互作用后的结果展现出来的结果。游戏场景当然也不仅仅是图像，它也是由后台的特殊数据结构或对象所确定。我们要想做出这个游戏，首先就要初始化这样一个对象，这也是Breakout类的基本功能：

```python
class Breakout:
    
    def __init__(self):            #__init__(self)=构造函数
        
        pygame.init()             
        
        self.screen = pygame.display.set_mode(SCREEN_SIZE)   #pygame.display.set_mode()就是创建一个游戏窗口
        pygame.display.set_caption("bricka a breakout clone by CTS")
        
        self.clock = pygame.time.Clock()  #控制游戏速度

        if pygame.font:             #字体控制
            self.font = pygame.font.Font(None,30)
        else:
            self.font = None
```

以上代码构建了一个Breakout类，并初始化了一个screen，注意screen只是一个容器，并不是真的显示屏幕。这意味着在screen上放些对象并不能直接显示出来，而要靠show或者display这样的方法来进行绘制，才能将screen上的图画真正显示出来。总之游戏的运行过程基本上就是:
__1.初始化各种对象__-》__2.根据游戏算法计算各个对象状态__-》__3.screen更新所有对象的位置__-》__4.显示新的screen内容__-》__重复2.__。

能放入screen这个容器中的对象也是有特定要求的，在pygame中，能显示的对象都是pygame.Rect的实例(具体参考[pygame教程](http://eyehere.net/2011/python-pygame-novice-professional-6/))，因此我们在设计Ball,Paddle,Brick类时，要继承这个基础类，才可以将其绘制到screen上。因为每个对象本质上都是Rect,因此必须实现自己的show方法才能绘制出不同的图像，据此我们建立如下的几个类：

In [2]:
class Ball(pygame.Rect):
    
    def __init__(self,x,y,r,color = BLACK):      #构造函数，x,y为球心坐标，r为半径,其中添加了颜色属性方便生成不同颜色的实例
        self.radius = r
        self.d = r*2
        pygame.Rect.__init__(self,x-r,y-r,x+r,y+r) #调用父类构造函数的一种方式也可以用,super(Ball,self).__init__(x-r,y-r,x+r,y+r)
        self.color = color
        
    def show(self,screen):
        
        pygame.draw.circle(screen, self.color, (self.x, self.y), self.radius)   
    
class Paddle(pygame.Rect):
    
    def __init__(self,rect,color = BLACK):     #用一个pygame.Rect对象来初始化
        
        pygame.Rect.__init__(self,rect)       #调用父类构造函数的一种方式也可以用,super(Paddle,self).__init__(rect)
        self.color = color
        
    def show(self,screen):
        
        pygame.draw.rect(screen, self.color, self)
         
class Brick(pygame.Rect):
    
    def __init__(self,rect,color=BLACK):
        
        pygame.Rect.__init__(self,rect)
        self.color=color
        
    def show(self,screen):
        
        pygame.draw.rect(screen, self.color, self)
    

接下来我们先考虑如何将这三种对象绘制到场景中去,在Breakout类中我们先添加游戏中的球、板、砖块作为成员变量,然后写一些初始化方法来进行初始化配置
```python
class Breakout:
    
    def __init__(self):
        
        pygame.init()
        
        self.screen = pygame.display.set_mode(SCREEN_SIZE)
        pygame.display.set_caption("bricka a breakout clone by CTS")
        
        self.clock = pygame.time.Clock()

        if pygame.font:
            self.font = pygame.font.Font(None,30)
        else:
            self.font = None
        self.init_game()

    def init_game(self):
        self.lives = 3
        self.score = 0
        self.state = STATE_BALL_IN_PADDLE
        self.ball = Ball((SCREEN_SIZE[0]/2),PADDLE_Y - BALL_RADIUS,BALL_RADIUS,BLACK)
        self.paddle=Paddle(pygame.Rect((SCREEN_SIZE[0]-PADDLE_WIDTH)/2,PADDLE_Y-PADDLE_HEIGHT,PADDLE_WIDTH,PADDLE_HEIGHT),BLACK)
       
  
        self.create_bricks()
        
           
    def create_bricks(self):
       
        y_ofs = BRICK_OFFSET_Y
        self.bricks = []
        for i in range(NBRICK_ROWS):
            x_ofs = 2
            for j in range(NBRICKS_PER_ROW):
                
                self.bricks.append(Brick(pygame.Rect(x_ofs,y_ofs,BRICK_WIDTH,BRICK_HEIGHT),RED))
                x_ofs += BRICK_WIDTH + BRICK_SEP
            y_ofs += BRICK_HEIGHT + BRICK_SEP

                 
    def update(self):
        self.paddle.show(self.screen)
        self.ball.show(self.screen)
        for brick in self.bricks:
            brick.show(self.screen) 
        
                                       
    def run(self):
        
        while 1:            
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    sys.exit()
                   
            self.clock.tick(50)  #限制帧率
            self.screen.fill(WHITE)
            
        
            self.update()      #更新screen
            pygame.display.flip() #双缓冲显示
```

然后生成Breakout实例并执行run()方法即可看到效果。

算法设计
===

现在我们已经可以将游戏的初始画面显示出来了，但只是显示静态图像而已，并没有任何功能。这是因为我们才刚刚完成程序设计中的第一步：数据结构设计，然后在此基础之上才能进行算法设计去实现具体的功能，而且数据结构的定义往往决定了用什么算法实现。这好比拍电影，真正要拍一部好戏，剧本再好没有合适的演员以及后台工作人员是不行的，写一个好程序，首要的工作不是急着去写算法实现功能，而是先进行建模，对程序进行清晰的描述，然后再考虑可以采用哪些算法。

现在我们的舞台上已经有了3种“演员”了，其中会动的只有球和球板两位。有两种实现思路，一种是给这些对象各自实现自己的移动算法，另一种是由Breakout类来控制对象的移动。鉴于这个程序很简单，我们采用后一种做法。首先考虑球的运动：球需要在场景边界，板以及砖块直接来回反弹运动，在游戏开始时，球有一个初始速度，分为x,y两个方向的分量速度就是每次更新画面时坐标的增量，只要在更新画面前根据速度算出新的坐标，就能实现运动了。


碰撞时速度的y分量取反即可得到反弹后速度。至于怎么知道碰撞的呢？其实是在每次运动后，检查球是否碰到边界或者其他对象，如果有，则反弹，没有就什么都不做。
```python
 def checkborder(self):    
        
        if self.ball.left <= 0:
            
            self.ball.left = 0
            self.ball_vel[0] = -self.ball_vel[0]
            
        elif self.ball.left >= MAX_BALL_X:
            
            self.ball.left = MAX_BALL_X
            self.ball_vel[0] = -self.ball_vel[0]     
               
        if self.ball.top < 0:
            
            self.ball.top = 0
            self.ball_vel[1] = -self.ball_vel[1]
            
        elif self.ball.top >= MAX_BALL_Y:      
                  
            self.ball.top = MAX_BALL_Y
            self.ball_vel[1] = -self.ball_vel[1] 
            
        if self.ball.top > self.paddle.top:   #掉到底部不反弹，生命值减一并复位
            self.lives -= 1
            if self.lives > 0:
                self.state = STATE_BALL_IN_PADDLE
            else:
                self.state = STATE_GAME_OVER  

def handle_collisions(self):
        
        for brick in self.bricks:
            if self.ball.colliderect(brick):
                self.score += brick.score
                self.ball_vel[1] = -self.ball_vel[1]
                self.bricks.remove(brick)        #消除砖块
                break
            
        if self.ball.colliderect(self.paddle):
            self.ball.top = PADDLE_Y - BALL_DIAMETER
            self.ball_vel[1] = -self.ball_vel[1]
        if len(self.bricks) == 0:
            self.state = STATE_WON #胜利了
```
           
具体代码：
```python
...
#class ball
class Ball(pygame.Rect):
    
    def __init__(self,left,top,r,color = BLACK):
        self.radius = r
        self.d = r*2
        pygame.Rect.__init__(self,left,top,self.d,self.d)
        self.color = color
        
    def show(self,screen):
        
         pygame.draw.circle(screen, self.color, (self.left + self.radius, self.top + self.radius), self.radius) 
         

#class breakout
...
  def set_ball_speed(self,vx,vy):
              
        if Random().random()<0.5: 
            vx = -vx    
        self.ball_vel = [vx,vy] 
        
  def init_game(self):
        self.lives = 3
        self.score = 0
        self.state = STATE_BALL_IN_PADDLE
        self.ball = Ball(int(SCREEN_SIZE[0]/2)-BALL_DIAMETER,PADDLE_Y - BALL_DIAMETER,BALL_RADIUS,BLACK)
        self.paddle=Paddle(pygame.Rect(SCREEN_SIZE[0]/2,PADDLE_Y,PADDLE_WIDTH,PADDLE_HEIGHT),BLACK)
       
        self.set_ball_speed( Random().randint(1,5),-5)
        self.create_bricks()
        
  def handle_collisions(self):
        
        for brick in self.bricks:
            if self.ball.colliderect(brick):
                self.score += brick.score
                self.ball_vel[1] = -self.ball_vel[1]
                self.bricks.remove(brick)        #消除砖块
                break
            
        if self.ball.colliderect(self.paddle):
            self.ball.top = PADDLE_Y - BALL_DIAMETER
            self.ball_vel[1] = -self.ball_vel[1]
        if len(self.bricks) == 0:
            self.state = STATE_WON #胜利了
           
  def checkborder(self):    
        
        if self.ball.left <= 0:
            
            self.ball.left = 0
            self.ball_vel[0] = -self.ball_vel[0]
            
        elif self.ball.left >= MAX_BALL_X:
            
            self.ball.left = MAX_BALL_X
            self.ball_vel[0] = -self.ball_vel[0]     
               
        if self.ball.top < 0:
            
            self.ball.top = 0
            self.ball_vel[1] = -self.ball_vel[1]
            
        elif self.ball.top >= MAX_BALL_Y:      
                  
            self.ball.top = MAX_BALL_Y
            self.ball_vel[1] = -self.ball_vel[1] 
            
        if self.ball.top > self.paddle.top:   #掉到底部不反弹，生命值减一并复位
            self.lives -= 1
            if self.lives > 0:
                self.state = STATE_BALL_IN_PADDLE
            else:
                self.state = STATE_GAME_OVER  
                
  def move_ball(self):
        self.handle_collisions()
        self.checkborder()
        self.ball.left += self.ball_vel[0]
        self.ball.top  += self.ball_vel[1]
 
  def run(self):
        while 1:            
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    sys.exit
                   
            self.clock.tick(50)
            self.screen.fill(WHITE)
            self.check_input() 
            self.move_ball()
            self.update()
   
            
            pygame.display.flip()
...
```

这样我们就能看到球动起来了，再加上对按键的响应以及状态转移控制，游戏基本完成，完整的代码如下：

In [ ]:
# -*- coding: UTF-8 -*-
'''


@author: cts
'''
import sys
import pygame
from random import Random


SCREEN_SIZE   = 400,600

# Object dimensions

BRICK_HEIGHT  = 10
PADDLE_WIDTH  = 50
PADDLE_HEIGHT = 10
BALL_DIAMETER = 14
BALL_RADIUS   = int(BALL_DIAMETER/2)
NBRICKS_PER_ROW = 10
NBRICK_ROWS = 10
BRICK_SEP = 4
BRICK_OFFSET_Y=70
BRICK_WIDTH  = int((SCREEN_SIZE[0] - (NBRICKS_PER_ROW) * BRICK_SEP) / NBRICKS_PER_ROW)
MAX_PADDLE_X = SCREEN_SIZE[0] - PADDLE_WIDTH
MAX_BALL_X   = SCREEN_SIZE[0] - BALL_DIAMETER
MAX_BALL_Y   = SCREEN_SIZE[1] - BALL_DIAMETER

# Paddle Y coordinate
PADDLE_Y = SCREEN_SIZE[1] - PADDLE_HEIGHT - 10

# Color constants
BLACK = (0,0,0)
WHITE = (255,255,255)
RED=(255,0,0)
GREEN=(0,255,0)
BLUE  = (0,0,255)
BRICK_COLOR = (200,200,0)
ORANGE=(255,153,0)
BROWN=(255,204,0)
# State constants
STATE_BALL_IN_PADDLE = 0
STATE_PLAYING = 1
STATE_WON = 2
STATE_GAME_OVER = 3

class Ball(pygame.Rect):
    
    def __init__(self,left,top,r,color = BLACK):
        self.radius = r
        self.d = r*2
        super(Ball,self).__init__(left,top,self.d,self.d)
        self.color = color
        
    def show(self,screen):
        
        pygame.draw.circle(screen, self.color, (self.left + self.radius,self.top + self.radius), self.radius)   
    
class Paddle(pygame.Rect):
    
    def __init__(self,rect,color = BLACK):
        
        pygame.Rect.__init__(self,rect)
        self.color = color
        
    def show(self,screen):
        
        pygame.draw.rect(screen, self.color, self)
         
class Brick(pygame.Rect):
    
    def __init__(self,rect,color=BLACK):
        
        pygame.Rect.__init__(self,rect)
        self.color=color
        
    def show(self,screen):
        
        pygame.draw.rect(screen, self.color, self)
        
class Breakout:
    
    def __init__(self):
        
        pygame.init()
        
        self.screen = pygame.display.set_mode(SCREEN_SIZE)
        pygame.display.set_caption("breakout by CTS")
        
        self.clock = pygame.time.Clock()

        if pygame.font:
            self.font = pygame.font.Font(None,30)
        else:
            self.font = None

    def init_game(self):
        self.lives = 3
        self.score = 0
        self.state = STATE_BALL_IN_PADDLE
        self.ball = Ball(int(SCREEN_SIZE[0]/2)-BALL_DIAMETER,PADDLE_Y - BALL_DIAMETER,BALL_RADIUS,BLACK)
        self.paddle=Paddle(pygame.Rect(SCREEN_SIZE[0]/2,PADDLE_Y,PADDLE_WIDTH,PADDLE_HEIGHT),BLACK)
       
        self.set_ball_speed( Random().randint(1,5),-5)
        self.create_bricks()
        
        
    def set_ball_speed(self,vx,vy):
        
        if Random().random()<0.5: 
            vx = -vx          
        self.ball_vel = [vx,vy] 
           
    def create_bricks(self):
        colorlist=[RED,ORANGE,BROWN,GREEN,BLUE]
        y_ofs = BRICK_OFFSET_Y
        self.bricks = []
        for i in range(NBRICKS_PER_ROW):
            x_ofs = 2
            for j in range(NBRICK_ROWS):
                self.bricks.append(Brick(pygame.Rect(x_ofs,y_ofs,BRICK_WIDTH,BRICK_HEIGHT),RED))
                x_ofs += BRICK_WIDTH + BRICK_SEP
            y_ofs += BRICK_HEIGHT + BRICK_SEP
            
    def move_ball(self):
        self.handle_collisions()
        self.checkborder()
        self.ball.left += self.ball_vel[0]
        self.ball.top  += self.ball_vel[1]
       
        
        
    def handle_collisions(self):
        for brick in self.bricks:
            if self.ball.colliderect(brick):
                self.score += 3
                self.ball_vel[1] = -self.ball_vel[1]
                self.bricks.remove(brick)
                break
        if self.ball.colliderect(self.paddle):
            self.ball.top = PADDLE_Y - BALL_DIAMETER
            self.ball_vel[1] = -self.ball_vel[1]
        if len(self.bricks) == 0:
            self.state = STATE_WON
            
    def checkborder(self):    
        
        if self.ball.left <= 0:
            
            self.ball.left = 0
            self.ball_vel[0] = -self.ball_vel[0]
            
        elif self.ball.left >= MAX_BALL_X:
            
            self.ball.left = MAX_BALL_X
            self.ball_vel[0] = -self.ball_vel[0]     
               
        if self.ball.top < 0:
            
            self.ball.top = 0
            self.ball_vel[1] = -self.ball_vel[1]
            
        elif self.ball.top >= MAX_BALL_Y:      
                  
            self.ball.top = MAX_BALL_Y
            self.ball_vel[1] = -self.ball_vel[1] 
            
        if self.ball.top > self.paddle.top:
            self.lives -= 1
            if self.lives > 0:
                self.state = STATE_BALL_IN_PADDLE
            else:
                self.state = STATE_GAME_OVER  
                 
    def check_input(self):
            keys = pygame.key.get_pressed()
        
            if keys[pygame.K_LEFT]:
                self.paddle.left -= 5
                if self.paddle.left < 0:
                    self.paddle.left = 0

            if keys[pygame.K_RIGHT]:
                self.paddle.left += 5
                if self.paddle.left > MAX_PADDLE_X:
                    self.paddle.left = MAX_PADDLE_X

            if keys[pygame.K_SPACE] and self.state == STATE_BALL_IN_PADDLE:
                
                self.set_ball_speed( Random().randint(1,5),-5)
                self.state = STATE_PLAYING
                
            elif keys[pygame.K_RETURN] and (self.state == STATE_GAME_OVER or self.state == STATE_WON):
                    self.init_game()
            
            if keys[pygame.K_ESCAPE]:
                exit()
                             
    def show_stats(self):
        if self.font:
            font_surface = self.font.render("SCORE: " + str(self.score) + " LIVES: " + str(self.lives), False, BLACK)
            self.screen.blit(font_surface, (100,5))

    def show_message(self,message):
        if self.font:
            size = self.font.size(message)
            font_surface = self.font.render(message,False, BLACK)
            x = (SCREEN_SIZE[0] - size[0]) / 2
            y = (SCREEN_SIZE[1] - size[1]) / 2
            self.screen.blit(font_surface, (x,y))  
            
    def update(self):
        self.paddle.show(self.screen)
        self.ball.show(self.screen)
        for brick in self.bricks:
            brick.show(self.screen)  
            
                                           
    def run(self):
        self.init_game()
        while 1:            
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    sys.exit
                   
            self.clock.tick(50)
            self.screen.fill(WHITE)
            self.check_input() 
            
            if self.state == STATE_PLAYING:
                
                self.move_ball()
                self.update()
                
            elif self.state == STATE_BALL_IN_PADDLE:
                
                self.ball.left = self.paddle.left + self.paddle.width / 2
                self.ball.top  = self.paddle.top - self.ball.height
                self.show_message("PRESS SPACE TO LAUNCH THE BALL")
                self.update()
                
            elif self.state == STATE_GAME_OVER:
               
                self.show_message("GAME OVER. PRESS ENTER")
                
            elif self.state == STATE_WON:
                
                self.show_message("YOU WON!")
                
                
            self.show_stats()    
            
            pygame.display.flip()
            
if __name__ == '__main__':
    
    game = Breakout()
    game.run()
       
        

进一步的任务
===

以上游戏只是实现了基本功能，其实一个真正打砖块的游戏还要更丰富。但是罗马不是一日建成的，那些复杂的游戏，也是从简单慢慢发展而来。你可以思考一下问题：
 - 目前砖块分数是固定的，如何做到每个砖块不同分值？
 - 目前砖块颜色都一样，如何实现多彩的砖块？
 - 目前球和砖都没有特殊效果，如何添加声效？
 - 如何实现球运动的真实的物理模型（考虑力的作用）？
 - 如何实现特殊功能的球（比如一次穿3块砖）以及砖头（藏有道具的砖头）？
 - 如何实现地图文件配置？如何实现关卡切换？
如果你试着去实现以上功能，你将会发现程序建模的重要性，并且真正了解到面向对象编程（OOP）的伟大意义。